In [1]:
# Reset variables
%reset


Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
# Import necessary libraries
import os
import json
import copy
import datetime
import numpy as np
import pandas as pd

from scipy import interpolate
from thin_plate_spline_warp import thin_plate_spline_warp_xyz, thin_plate_spline_warp_xz

In [3]:
name = "OD1599_NU"

In [4]:
# Define some helper functions
def create_workspace_folders():
    date = datetime.datetime.now()
    workspace = "workspace\{}-{}".format(date.strftime("%Y_%m_%d"), name)

    create_folders = ['workspace', workspace]
    # create workspace
    for folder in create_folders:
        if not os.path.exists(folder):
            os.makedirs(folder)
    
    return workspace

def moving_average(a, window_size):
    new_a = []
    half_window = round(window_size/2)
    for idx, val in enumerate(a):
        if val == 0:
            new_a.append(0)
            continue
            
        start_idx = max(idx-half_window, 0)
        end_idx = min(idx+half_window+1, a.shape[0])
        a_segment = a[start_idx:end_idx]
        
        a_seg_mean = np.nanmean(np.where(a_segment!=0, a_segment, np.nan)) # nonzero mean
        new_a.append(a_seg_mean)
        
    return new_a

def hampel_filter(input_series, window=5, n_sigmas=1):
    # ensure the data is flattened
    input_series = np.array(input_series).flatten()
    
    # returns filtered timepoints and coordinates along
    n = len(input_series)
    k = 1.4826 # scale factor for Gaussian distribution
    
    outliers_idxs = []
    outliers_filtered = []
    extended_series = np.pad(input_series, (window, window), 'reflect')
    filtered_series = extended_series.copy()
    n_ex = len(extended_series)
    for i in range(window_size, n_ex - window_size):
        x0 = np.median(
            extended_series[(i - window_size):(i + window_size)])
        S0 = k * np.median(np.abs(
            extended_series[(i - window_size):(i + window_size)] - x0))
        if (np.abs(extended_series[i] - x0) > n_sigmas * S0):
            filtered_series[i] = x0 # replaces outlier with median
            outliers_idxs.append(i-window) # logs outlier index
            outliers_filtered.append(x0) # the value that replaces outlier
    
    filtered_series = filtered_series[window:len(filtered_series)-window]
    
    return filtered_series, outliers_idxs, outliers_filtered

def write_error(workspace_folderpath, error, write_type="w"):    
    error_path = os.path.join(workspace_folderpath, 'errors.txt')
    
    with open(error_path, write_type) as f:
        f.write(error)

In [5]:
# Setup
# Create name + workspace folders
workspace_folderpath = create_workspace_folders()
print('Workspace folderpath: {}'.format(workspace_folderpath))

# Load necessary file structures via config file
with open('config.json') as f:
    config = json.load(f)

# Clear error report
write_error(workspace_folderpath, '')


Workspace folderpath: workspace\2020_08_24-OD1599_NU


In [6]:
# STEP 1: Import all the necessary data
def parse_mipav_data(config, cell_key, strain_folderpath):
    # define output variables
    valid_seam_cells = set(config['settings']['interpolation']['seam_cells_on'].keys())
    seam_cell_output = {}
    annotation_output = {} # structured cell_name: pandas dataframe (timepoint, x, y, z)
    errors = [] # note potential errors in volumes
    
    # determine default or options specific to strain
    folderpaths = config['settings']['folderpaths']
    for folder_data in folderpaths.keys():
        try:
            folderpaths[folder_data] = cell_key['folderpaths'][folder_data]
            print('Using specific {} folder structure'.format(folder_data))
        except:
            # print('Using default {} folder structure'.format(folder_data))
            pass
    
    # find the numbers of the folders
    start_vol = int(cell_key['start'])
    end_vol = int(cell_key['end'])
    all_folderpaths = folderpaths.copy()
    for vol_idx, vol_num in enumerate(range(start_vol, end_vol+1)):
        # ignore outliers
        if vol_num in cell_key['outliers']:
            continue
        
        # define the specific volume number folder
        vol_folder = all_folderpaths['data_folderpath'].replace('#', str(vol_num))
        vol_folderpath = os.path.join(
            strain_folderpath, all_folderpaths['side'], vol_folder)
        
        # check if the folder exists
        if not os.path.isdir(vol_folderpath):
            continue
        
        # define individual, full filepaths from drive
        all_filepaths_folderpaths = {}
        for full_folderpaths_key in all_folderpaths.keys():
            if full_folderpaths_key != "side" and \
                full_folderpaths_key != "data_folderpath":
                all_filepaths_folderpaths[full_folderpaths_key] = os.path.join(
                    vol_folderpath, all_folderpaths[full_folderpaths_key])
                # print(all_filepaths_folderpaths[full_folderpaths_key])

                # check to see if exists
                if not os.path.isfile(all_filepaths_folderpaths[full_folderpaths_key]):
                    error_msg = "FILE DOES NOT EXIST: {}".format(
                        all_filepaths_folderpaths[full_folderpaths_key])
                    errors.append(error_msg)
                    print(error_msg)
        
        ## get individual data by cell in pandas format
        try:
            # import twisted seam cells           
            tw_seam_cells_fp = all_filepaths_folderpaths['twisted_seam_cells']
            tw_seam_cells = pd.read_csv(tw_seam_cells_fp)            
            # import twisted annotations
            tw_annotations_fp = all_filepaths_folderpaths['twisted_annotations']
            tw_annotations = pd.read_csv(tw_annotations_fp)
            # import straighted seam cells
            st_seam_cells_fp = all_filepaths_folderpaths['straightened_seam_cells']
            st_seam_cells = pd.read_csv(st_seam_cells_fp)
            # import straightened annotations
            st_annotations_fp = all_filepaths_folderpaths['straightened_annotations']
            st_annotations = pd.read_csv(st_annotations_fp)
        except:
            continue # file errors should already be logged.
            
        # ERROR CHECKING: Check for file content ----------------------
        if st_seam_cells.size == 0: # check if empty
            error_msg = "DATA ERROR: Empty seam cell file at {}".format(st_seam_cells_fp)
            errors.append(error_msg)
            print(error_msg)
        if st_annotations.size == 0: # check if empty
            error_msg = "DATA ERROR: Empty annotation file at {}".format(st_annotations_fp)
            errors.append(error_msg)
            print(error_msg)
        data_file_list = [ # use to check values of files
            (tw_seam_cells, tw_seam_cells_fp), (tw_annotations, tw_annotations_fp),
            (st_seam_cells, st_seam_cells_fp), (st_annotations, st_annotations_fp)
        ]
        for data, fp in data_file_list:
            coordinates = data[['x_voxels', 'y_voxels', 'z_voxels']]
            cell_ids = data['name'].values.tolist()
            for cell_idx, cell_coord in coordinates.iterrows():
                if not cell_coord.equals(cell_coord.astype(float)):
                    error_msg = "DATA ERROR: Non-float found for cell {} in {}".format(
                        cell_ids[cell_idx], fp)
                    errors.append(error_msg)
                    print(error_msg)            
        seam_cells_mismatch = list(set(st_seam_cells['name'].values.tolist()) - 
                                   set(tw_seam_cells['name'].values.tolist()))
        annotations_mismatch = list(set(st_annotations['name'].values.tolist()) - 
                                   set(tw_annotations['name'].values.tolist())) 
        if seam_cells_mismatch:
            error_msg = "DATA ERROR: Mis-match between seam cells ({}) found in {} and {}".format(
                ", ".join(seam_cells_mismatch), st_seam_cells_fp, tw_seam_cells_fp)
            errors.append(error_msg)
            print(error_msg)
        if annotations_mismatch:
            error_msg = "DATA ERROR: Mis-match between annotated cells ({}) found in {} and {}".format(
                ", ".join(annotations_mismatch), st_annotations_fp, tw_annotations_fp)
            errors.append(error_msg)
            print(error_msg)
        # check if extra seam cells exist/mismatch with settings
        extra_seam_cells = list(set(st_seam_cells['name'].values.tolist()) - valid_seam_cells)
        if extra_seam_cells: # check if there are extra seam cells
            # save old one as _bak.csv, save new as straightened_seamcells.csv
            # st_seam_cells.to_csv(st_seam_cells_fp.replace('.csv', '_bak.csv'), index=False)
            for extra_seam_cell in extra_seam_cells:
                st_seam_cells = st_seam_cells[st_seam_cells['name'] != extra_seam_cell]
            # st_seam_cells.to_csv(st_seam_cells_fp, index=False)
            error_msg = "DATA WARNING: Ignoring extra seam cells ({}) found in {}".format(
                ", ".join(extra_seam_cells), st_seam_cells_fp)
            errors.append(error_msg)
            print(error_msg)
        # END FILE ERROR CHECK -----------------------------------------------------------

        # SEAM CELLS: combine into a single data structure and catch data errors
        # go through each row of the file and log the appropriate information
        for idx, seam_row in st_seam_cells.iterrows():
            seam_cell_name = seam_row['name'].upper() # seam cells are upper case? just stay consistent
            if seam_cell_name not in seam_cell_output.keys():
                seam_cell_output[seam_cell_name] = {}
                seam_cell_output[seam_cell_name]['timepoints'] = []
                seam_cell_output[seam_cell_name]['coordinates'] = []

            # append to appropriate seam cell
            if vol_num not in seam_cell_output[seam_cell_name]['timepoints']:
                seam_cell_coords = seam_row[['x_voxels','y_voxels','z_voxels']].values.flatten().tolist()
                seam_cell_output[seam_cell_name]['coordinates'].append(seam_cell_coords)
                seam_cell_output[seam_cell_name]['timepoints'].append(vol_num)

        # ANNOTATIONS: combine into a single data structure and catch data errors
        try:
            for cell_id in cell_key['mapping'].keys():
                cell_name = cell_key['mapping'][cell_id].lower()
                if cell_name not in annotation_output.keys():
                    annotation_output[cell_name] = {}
                    annotation_output[cell_name]['timepoints'] = []
                    annotation_output[cell_name]['coordinates'] = []

                # check if the cell name exists in volume
                # print(vol_folderpath, st_annotations['name'], cell_id)
                cell_row = st_annotations.loc[st_annotations['name'] == cell_id]
                if not cell_row.empty:
                    # try to catch a few errors
                    if cell_row.shape[0] > 1:
                        error_msg = "DATA ERROR: Ignoring identical cell IDs ({}) for cell {} in {}".format(
                            cell_id, cell_name, st_annotations_fp)
                        errors.append(error_msg)
                        print(error_msg)
                        continue
                    
                    # if there is nothing wrong, then proceed
                    if vol_num not in annotation_output[cell_name]['timepoints']:
                        cell_coords = cell_row[['x_voxels','y_voxels','z_voxels']].values.flatten().tolist()
                        annotation_output[cell_name]['coordinates'].append(cell_coords)
                        annotation_output[cell_name]['timepoints'].append(vol_num)
        except:
            error_msg = "DATA ERROR: Failure to read file {}".format(st_annotations_fp)
            errors.append(error_msg)
            print(error_msg)
                
    return seam_cell_output, annotation_output, errors

def convert_cell_key_csv2json(csv_filepath):
    cell_key_json = {}
    # load in csv file
    cell_key = pd.read_csv(csv_filepath, header=None, engine='python')
    
    ## get necessary information
    cell_key_json['name'] = str(cell_key.iloc[0,0])
    cell_key_json['start'] = int(cell_key.iloc[1,0])
    cell_key_json['end'] = int(cell_key.iloc[1,1])
    try:
        cell_key_json['outliers'] = cell_key.iloc[2].astype(int).values.tolist()
    except:
        cell_key_json['outliers'] = [] # if there are none
    
    # grab mapping
    cell_key_json['mapping'] = {}
    for row_idx in range(3, cell_key.shape[0]): # mapping starts row idx 3
        cell_id = str(cell_key.iloc[row_idx,0])
        cell_name = str(cell_key.iloc[row_idx,1])
        cell_key_json['mapping'][cell_id] = cell_name

    return cell_key_json
    
def get_cell_key(pos_folderpath):
    # return the cell key in dict/json format and convert any existing 
    # cell key csv into json if the json version doesn't exist.
    cell_key_filepath_csv = os.path.join(pos_folderpath, 'CellKey.csv')
    cell_key_filepath_json = os.path.join(pos_folderpath, 'cell_key.json')
    
    # if the json exists, use it
    if os.path.isfile(cell_key_filepath_json):
        with open(cell_key_filepath_json) as f:
            return json.load(f)
    elif os.path.isfile(cell_key_filepath_csv):
        cell_key_json = convert_cell_key_csv2json(cell_key_filepath_csv)
        cell_key_json_dump = json.dumps(cell_key_json, sort_keys=True, indent=4)
        with open(cell_key_filepath_json, "w") as f: 
            f.write(cell_key_json_dump)
        return cell_key_json
    else:
        cell_key_error_msg = 'Missing cell key: {}'.format(cell_key_filepath_json)
        print(cell_key_error_msg)
        return None
                
strain_info = config['data']['strains']
compiled_data = {}
for strain in strain_info:
    if strain['include']:
        if strain['name'] not in compiled_data.keys():
            compiled_data[strain['name']] = {}
                
        for pos_folderpath in strain['folderpaths']:
            print('CURRENTLY LOADING: {}'.format(pos_folderpath))
            # find and load cell key
            cell_key = get_cell_key(pos_folderpath)
            if not cell_key:
                continue
                
            # get data
            seam_cells, annotations, strain_errors = parse_mipav_data(
                config, cell_key, pos_folderpath)
            
            # combine data
            pos_name = cell_key['name']
            compiled_data[strain['name']][pos_name] = {}
            compiled_data[strain['name']][pos_name]['cell_key'] = cell_key
            compiled_data[strain['name']][pos_name]['seam_cells'] = seam_cells
            compiled_data[strain['name']][pos_name]['annotations'] = annotations
            # compiled_data[strain['name']][pos_name]['errors'] = strain_errors
            
            # write necessary errors to file
            if strain_errors:
                write_error(workspace_folderpath, '\n'.join(strain_errors) + '\n', 'a')

# save information as intermediate step in workspace
compiled_json = json.dumps(compiled_data, sort_keys=True, indent=4)
compiled_json_filepath = os.path.join(
    workspace_folderpath, '1_compiled_data.json')
with open(compiled_json_filepath, "w") as f: 
    f.write(compiled_json)

CURRENTLY LOADING: Y:\RyanC\Cell Tracking Project\OD1599_NU\OD1599_MostRecent\120619_Pos2\Decon_reg
CURRENTLY LOADING: Y:\RyanC\Cell Tracking Project\OD1599_NU\OD1599_MostRecent\112719_Pos3\Decon_Reg
CURRENTLY LOADING: Y:\RyanC\Cell Tracking Project\OD1599_NU\OD1599_MostRecent\112619_Pos0\Decon_reg


In [7]:
# STEP 2: Check for outliers, uses parameters defined in config.json

window_size = config['settings']['outlier_removal']['window_size']
n_stdev = config['settings']['outlier_removal']['n_stdev']
compiled_data_no_outliers = copy.deepcopy(compiled_data)
for strain in compiled_data.keys():
    for pos in compiled_data[strain].keys():

        # determine outliers for both seam cells and annotations
        for cells_type in ['seam_cells', 'annotations']:
            for cell in compiled_data[strain][pos][cells_type].keys():
                coordinates = np.array(compiled_data[strain][pos][cells_type][cell]['coordinates'])

                # separate by x,y,z (index 0-2) and filter
                outlier_idxs = {} # set automatically removes repeats
                for dim_idx in range(3):
                    coord_dim_data = copy.deepcopy(coordinates[:, dim_idx])
                    data_filtered, outlier_idx, outlier_val = hampel_filter(coord_dim_data, 
                            n_sigmas=n_stdev, window=window_size)
                    coordinates[:, dim_idx] = data_filtered

                # replace time series outliers using median
                compiled_data_no_outliers[strain][pos][cells_type][cell]['coordinates'] = coordinates.tolist()

print('Step 2 Completed.')
# save information as intermediate step in workspace
compiled_json = json.dumps(compiled_data_no_outliers, sort_keys=True, indent=4)
compiled_json_filepath = os.path.join(
    workspace_folderpath, '2_compiled_data_no_outliers.json')
with open(compiled_json_filepath, "w") as f: 
    f.write(compiled_json)
print('Step 2 Data Logged.')

Step 2 Completed.
Step 2 Data Logged.


In [8]:
# STEP 3: Interpolate each to appropriate time scale
compiled_data_interp = copy.deepcopy(compiled_data_no_outliers)
seam_cells_on = config['settings']['interpolation']['seam_cells_on'] # in minutes
total_len = config['settings']['interpolation']['total_min'] # in minutes
interp_method = config['settings']['interpolation']['method']
min_timepoints_required = config['settings']['interpolation']['min_timepoints_required']
new_timepoints = np.linspace(0, total_len)

for strain in compiled_data_no_outliers.keys():
    for pos in compiled_data_no_outliers[strain].keys():
        
        cell_key = compiled_data_no_outliers[strain][pos]['cell_key']
        # interpolate for both seam cells and annotations
        for cells_type in ['seam_cells', 'annotations']:
            for cell in compiled_data_no_outliers[strain][pos][cells_type].keys():
                timepoints = np.array(compiled_data_no_outliers[strain][pos][cells_type][cell]['timepoints'])
                coordinates = np.array(compiled_data_no_outliers[strain][pos][cells_type][cell]['coordinates'])
                
                # CHECKS if there are a sufficient number of timepoints. if not, raise error.
                if timepoints.size < min_timepoints_required:
                    error_msg = "DATA ERROR: Insufficient timepoints in {}/{} for cell {} where only {} (volumes {}), less than the required {}, exist.".format(
                        strain, pos, cell, str(len(timepoints)), 
                        ",".join(timepoints.astype(str).tolist()), str(min_timepoints_required))
                    # compiled_data_interp[strain][pos]['errors'].append(error_msg)
                    print(error_msg)
                    write_error(workspace_folderpath, error_msg + '\n', 'a')
                    continue
                
                # handle seam cells and annotations differently because 
                # some seam cells appear after twitching begins
                if cells_type == 'seam_cells':
                    # determine if strain starts after designated seam cell on
                    target_percent = seam_cells_on[cell]
                    # target_starting_idx = int(round(target_percent * total_len)) # after scaling
                    
                    # find the closest volume to the target
                    vol_len = cell_key['end'] - cell_key['start']
                    og_target = round(vol_len * target_percent + cell_key['start'])
                    og_target_idx = np.argmin(np.abs(timepoints - og_target)).astype(int) # in original time
                    seam_cell_on_vol = timepoints[og_target_idx]
                    seam_cell_on_percent = (seam_cell_on_vol - cell_key['start'])/vol_len # calculate what the actual percentage is
                    
                    # determine what to do if greater than or less than target percentage
                    og_starting_idx = og_target_idx
                    if seam_cell_on_percent > target_percent: # starts after expected
                        starting_idx = int(round(seam_cell_on_percent * total_len))
                    elif seam_cell_on_percent <= target_percent: # starts before expected
                        starting_idx = int(round(target_percent * total_len))
                    
                    """
                    if cell == "QL":
                        print(cell, timepoints)
                        print(seam_cells_on[cell], seam_cell_on_percent, og_target, 
                              cell_key['start'], og_starting_idx, starting_idx)"""
                    
                elif cells_type == 'annotations':
                    starting_idx = 0
                    og_starting_idx = 0
                
                new_coordinates = np.zeros((total_len, 3))
                new_timepoints = np.arange(total_len)

                # interpolate one dimension at a time
                for dim_idx in range(3):
                    # crop the original data to the appropriate length, e.g. Q: top 25% of volumes
                    cell_timepoints = timepoints[og_starting_idx:].copy()
                    coord_dim_data = coordinates[og_starting_idx:, dim_idx].copy()

                    # rescale time points to range, 0 to end, rather than e.g. 8 to 54
                    cell_sp_rescaled = cell_timepoints - min(cell_timepoints)
                    cell_sp_rescaled = cell_sp_rescaled/max(cell_sp_rescaled) * (total_len - starting_idx)
                
                    interp = interpolate.interp1d(cell_sp_rescaled, coord_dim_data,
                                     kind=interp_method) # get interp as if from 0, but shift below
                    
                    # new time scale for specific length (might be part of total)  
                    cell_sp_timepoints = np.arange(total_len - starting_idx)
                    interped_coords = interp(cell_sp_timepoints)
                    new_coordinates[starting_idx:, dim_idx] = interped_coords # shifted

                # replace time series outliers using median
                compiled_data_interp[strain][pos][cells_type][cell]['coordinates'] = new_coordinates.tolist()
                compiled_data_interp[strain][pos][cells_type][cell]['timepoints'] = new_timepoints.tolist()
                
# save information as intermediate step in workspace
print('Step 3 Completed.')
compiled_json = json.dumps(compiled_data_interp, sort_keys=True, indent=4)
compiled_json_filepath = os.path.join(
    workspace_folderpath, '3_compiled_data_interpolation.json')
with open(compiled_json_filepath, "w") as f: 
    f.write(compiled_json)
print('Step 3 Data Logged.')

Step 3 Completed.
Step 3 Data Logged.


In [9]:
# STEP 4: Generate seam cell warping model
exclude_seam_cells = ['QL', 'QR']
seam_cell_strains = config['data']['seam_cells']
smoothing_window = config['settings']['smoothing']['window_size']

# check if it's a pre-generated warping model
pre_generated_model = False
if '.json' in seam_cell_strains[0]:
    pre_generated_model = True
    with open(seam_cell_strains[0]) as f:
        output_json = json.load(f)

    print("Using pre-generated warping model: {}".format(seam_cell_strains[0]))
else: # generate new warping model based on the strains given
    # go through files and look for the strain names
    warp_strain_names = []
    for seam_strain_folder in seam_cell_strains:
        cell_key_filepath_json = os.path.join(seam_strain_folder, 'cell_key.json')

        # if the json exists, use it
        if os.path.isfile(cell_key_filepath_json):
            with open(cell_key_filepath_json) as f:
                strain_cell_key = json.load(f)
                warp_strain_names.append(strain_cell_key['name'])

    # if not warp strain names, use a cached warping model
    seam_warp_model_folderpath = config['data']['seam_cells']
    print("Using the following strains for warping: {}".format(", ".join(warp_strain_names)))

    # first combine all the necessary data for seam cells
    warp_model_by_cell = {}
    for strain in compiled_data_interp.keys():
        for pos in compiled_data_interp[strain].keys():

            if pos not in warp_strain_names:
                continue

            for cell in compiled_data_interp[strain][pos]['seam_cells'].keys():
                if cell in exclude_seam_cells:
                    continue 

                if cell not in warp_model_by_cell.keys():
                    warp_model_by_cell[cell] = []

                seam_cell_coordindates = np.array(
                    compiled_data_interp[strain][pos]['seam_cells'][cell]['coordinates'])
                warp_model_by_cell[cell].append(
                    seam_cell_coordindates.tolist())

    # average all coordinates by cell for warping model
    for cell in warp_model_by_cell.keys():
        # average all the coordinates, step 5
        cell_coords = np.array(warp_model_by_cell[cell])
        warp_model_by_cell[cell] = np.average(warp_model_by_cell[cell], axis=0) # full coordinates

        # smooth, step 7
        coordinates = warp_model_by_cell[cell]
        for dim_idx in range(3):
            coord_dim_data = coordinates[:, dim_idx].copy()
            data_smoothed = moving_average(coord_dim_data, smoothing_window)
            coordinates[:, dim_idx] = data_smoothed

        # convert to list
        warp_model_by_cell[cell] = coordinates.tolist() # full coordinates

    # convert to all cells for each timepoint
    total_len = config['settings']['interpolation']['total_min'] # in minutes
    new_timepoints = np.arange(0, total_len)
    seam_cells_on = config['settings']['interpolation']['seam_cells_on']
    sorted_seam_cells = sorted(warp_model_by_cell.keys()) # maintain some order
    warp_model_by_timepoint = []
    for timepoint in new_timepoints.astype(int):
        timepoint_coords_by_cell = []
        for seam_cell in sorted_seam_cells:
            all_timepoint_coords = np.array(warp_model_by_cell[seam_cell])
            seam_cell_coords = all_timepoint_coords[timepoint, :].tolist()
            timepoint_coords_by_cell.append(seam_cell_coords)

        warp_model_by_timepoint.append(timepoint_coords_by_cell)

    # create output structure
    output_json = {}
    output_json['seam_cells'] = sorted_seam_cells
    output_json['coordinates'] = warp_model_by_timepoint

print('Step 4 Completed.')
# save information as intermediate step in workspace
warping_model = output_json
compiled_json = json.dumps(output_json, sort_keys=True, indent=4)
compiled_json_filepath = os.path.join(
    workspace_folderpath, '4_seam_cell_warping_model.json')
with open(compiled_json_filepath, "w") as f: 
    f.write(compiled_json)
print('Step 4 Data Logged.')

Using pre-generated warping model: Y:\RyanC\model_building_code\resources\warping_models\jul-11-2019_warping_model.json
Step 4 Completed.
Step 4 Data Logged.


In [10]:
# STEP 5: Warp strains to warping model
# we need to do this first because seam cell information
timepoints = np.arange(0, total_len).astype(int).tolist()
compiled_data_warped = copy.deepcopy(compiled_data_interp)
for strain in compiled_data_interp.keys():
    for pos in compiled_data_interp[strain].keys():
        print('Warping {}, Position {}'.format(strain, pos))
        for timepoint in timepoints:
            # get warp from/to model at timepoint (the seam cells)
            pos_seam_cells = compiled_data_interp[strain][pos]['seam_cells']
            warp_to_seam_cell_names = warping_model['seam_cells']
            warp_to_seam_cell_coords = warping_model['coordinates']
            warp_to_at_timepoint = warp_to_seam_cell_coords[timepoint] # contains all warping data
            
            # however, we might not need/have all the data so here we match what we have
            warp_to = []
            warp_from = []
            for warp_seam_cell_idx, warp_to_seam_cell_name in enumerate(warp_to_seam_cell_names):
                if warp_to_seam_cell_name in pos_seam_cells.keys():
                    # add to warp to
                    warp_to_coords = warp_to_at_timepoint[warp_seam_cell_idx]
                    warp_to.append(warp_to_coords)
                    
                    # add to warp from
                    all_time_coords = pos_seam_cells[warp_to_seam_cell_name]['coordinates']
                    warp_from_timepoint_coord = all_time_coords[timepoint]
                    warp_from.append(warp_from_timepoint_coord)
            
            warp_to = np.array(warp_to)
            warp_from = np.array(warp_from)
            # print('from', np.array(warp_from))
            # print('to', np.array(warp_to))
            
            # warp each cell at timepoint, including the seam cells themselves
            for cell_type in ['seam_cells', 'annotations']:
                # get cell order for warping
                sorted_cell_names = compiled_data_interp[strain][pos][cell_type].keys()
                ordered_coord_list = []
                for cell_name in sorted_cell_names:
                    old_coords = [compiled_data_interp[strain][pos][cell_type][cell_name]['coordinates'][timepoint]]
                    old_coords = np.array(old_coords).flatten().tolist() # ensure correct data shape
                    ordered_coord_list.append(old_coords)
                ordered_coord_list = np.array(ordered_coord_list)
                
                # warping
                if cell_type == 'seam_cells':
                    new_coords = thin_plate_spline_warp_xyz(warp_from, warp_to, ordered_coord_list)
                else:
                    new_coords = thin_plate_spline_warp_xz(warp_from, warp_to, ordered_coord_list)
                new_coords = new_coords.tolist()
                
                # assigning
                for cell_idx, cell_name in enumerate(sorted_cell_names):
                    # if the old coordinates used to be 0 (ignored), then keep it that way
                    old_coords = compiled_data_interp[strain][pos][cell_type][cell_name]['coordinates'][timepoint]
                    
                    if old_coords == [0, 0, 0]:
                        assign_coords = old_coords
                    else:
                        assign_coords = new_coords[cell_idx]
                        
                    compiled_data_warped[strain][pos][cell_type][cell_name]['coordinates'][timepoint] = \
                        assign_coords
    
print('Step 5 Completed.')
# save information as intermediate step in workspace
output_json = compiled_data_warped
compiled_json = json.dumps(output_json, sort_keys=True, indent=4)
compiled_json_filepath = os.path.join(
    workspace_folderpath, '5_compiled_data_warped.json')
with open(compiled_json_filepath, "w") as f: 
    f.write(compiled_json)
print('Step 5 Data Logged.')

Warping OD1599_NU, Position OD1599_NU_1206_Pos2
0.0 112.0
0.0 110.45
0.0 108.90000000000002
0.0 107.34999999999998
0.0 105.8
0.0 104.46428571428571
0.0 103.35714285714286
0.0 102.25
0.0 101.14285714285714
0.0 100.03571428571428
0.0 100.0
0.0 100.0
0.0 100.0
0.0 100.0
0.0 100.59999999999998
0.0 101.92857142857142
0.0 103.25714285714284
0.0 104.58571428571432
0.0 105.91428571428568
0.0 102.27142857142856
0.0 98.28571428571428
0.0 94.3
0.0 90.31428571428572
0.0 88.46428571428572
0.0 89.57142857142858
0.0 90.67857142857142
0.0 91.78571428571426
0.0 92.89285714285714
0.0 93.0
0.0 93.0
0.0 93.0
0.0 93.0
0.0 93.42857142857142
0.0 94.53571428571426
0.0 95.64285714285714
0.0 96.75
0.0 97.85714285714286
0.0 98.0
0.0 98.0
0.0 98.0
0.0 98.0
0.0 96.97857142857143
0.0 94.09999999999998
0.0 91.22142857142856
0.0 88.34285714285714
0.0 85.46428571428574
0.0 85.55714285714286
0.0 86.22142857142856
0.0 86.8857142857143
0.0 87.54999999999998
0.0 88.57142857142858
0.0 90.34285714285714
0.0 92.1142857142857

-0.0012772584615384614 76.15000000000002
-0.0013015873846153846 74.59999999999998
-0.001326860923076923 73.04999999999998
-0.0013531353846153846 71.49999999999999
-0.0013804713846153848 71.3
-0.001408934769230769 71.74285714285716
-0.0014385966153846153 72.1857142857143
-0.0014695341538461537 72.62857142857142
-0.0015018313846153846 72.96428571428572
-0.0015355806153846154 72.74285714285716
-0.0015708812307692309 72.52142857142854
-0.0016078430769230771 72.29999999999998
-0.0016465864615384615 72.07857142857142
-0.001671810769230769 72.0
-0.0016244726153846153 72.0
-0.0014953101538461538 72.0
-0.0012777778461538461 72.0
-0.0009646120000000001 72.14285714285714
-0.0005477307692307692 73.25
-0.000205314 74.35714285714286
-2.4875621538461535e-05 75.46428571428574
0.0 76.5714285714286
0.0 77.0
0.0 77.0
0.0 77.0
0.0 77.0
0.0 77.15000000000005
0.0 78.70000000000002
0.0 80.25000000000001
0.0 81.79999999999998
0.0 83.34999999999997
0.0 83.8714285714286
0.0 83.65000000000002
0.0 83.428571428571

-0.0009172258461538462 73.42857142857136
-0.0009172258461538462 71.28571428571422
-0.0009172258461538462 71.14285714285725
-0.0009172258461538462 74.00000000000004
-0.0009172258461538462 76.85714285714288
-0.0009172258461538462 79.71428571428572
-0.0009172258461538462 81.6666666666667
-0.0009172258461538462 80.00000000000001
-0.0009172258461538462 78.33333333333333
-0.0009172258461538462 76.66666666666669
-0.0009172258461538462 75.0
-0.0009172258461538462 75.23809523809523
-0.0009172258461538462 75.47619047619045
-0.0009172258461538462 75.71428571428574
-0.0009172258461538462 75.95238095238095
-0.0009172258461538462 76.0
-0.0009172258461538462 76.0
-0.0009172258461538462 76.0
-0.0009172258461538462 76.0
-0.0009172258461538462 76.14285714285714
-0.0009172258461538462 76.38095238095237
-0.0009172258461538462 76.61904761904762
-0.0009172258461538462 76.85714285714286
-0.0009172258461538462 76.90476190476191
-0.0009172258461538462 76.66666666666669
-0.0009172258461538462 76.42857142857142


0.0 80.26666666666667
0.0 81.0
0.0 79.53333333333333
0.0 78.06666666666665
0.0 76.59999999999998
0.0 75.1333333333333
0.0 73.66666666666666
0.0 74.70000000000005
0.0 77.81666666666669
0.0 80.93333333333337
0.0 84.05000000000001
0.0 87.1666666666667
0.0 89.71666666666667
0.0 86.59999999999998
0.0 83.48333333333335
0.0 80.3666666666667
0.0 77.25000000000003
0.0 74.13333333333338
0.0 73.0
0.0 73.0
0.0 73.0
0.0 73.0
0.0 73.0
0.0 73.3666666666667
0.0 75.38333333333337
0.0 77.40000000000003
-8.389261538461538e-06 79.41666666666671
-2.7032064615384615e-05 81.43333333333337
-5.59284e-05 83.45000000000005
-9.507830769230768e-05 83.73333333333333
-0.00014448172307692306 83.36666666666665
-0.00020413870769230767 83.0
-0.00027404923076923077 82.63333333333335
-0.0003542132307692308 82.26666666666667
-0.0004446307692307692 82.09999999999998
-0.0005453021538461539 82.46666666666667
-0.0006562267692307692 82.83333333333331
-0.0007494406153846154 83.2
-0.0008221476923076923 83.56666666666665
-0.000874

In [11]:
# STEP 6: Reformat data to average points
step_data = copy.deepcopy(compiled_data_warped)
data_by_cell = {'seam_cells':{}, 'annotations':{}}
for strain in step_data.keys():
    for pos in step_data[strain].keys():
        for cell_type in ['seam_cells', 'annotations']:
            for cell in step_data[strain][pos][cell_type].keys():
                if cell not in data_by_cell.keys():
                    data_by_cell[cell_type][cell] = []

                cell_coordindates = np.array(
                    step_data[strain][pos][cell_type][cell]['coordinates'])
                data_by_cell[cell_type][cell].append(
                    cell_coordindates.tolist())
            
# average all coordinates by cell
for cell_type in ['seam_cells', 'annotations']:
    # seam cells should just result in the average warping model
    for cell in data_by_cell[cell_type].keys():
        # average all the coordinates
        cell_coords = np.array(data_by_cell[cell_type][cell])
        data_by_cell[cell_type][cell] = np.average(data_by_cell[cell_type][cell], axis=0).tolist()

print('Step 6 Completed.')
# save information as intermediate step in workspace
output_json = data_by_cell
compiled_json = json.dumps(output_json, sort_keys=True, indent=4)
compiled_json_filepath = os.path.join(
    workspace_folderpath, '6_cell_coordinates_by_timepoint.json')
with open(compiled_json_filepath, "w") as f: 
    f.write(compiled_json)
print('Step 6 Data Logged.')

Step 6 Completed.
Step 6 Data Logged.


In [12]:
# STEP 7: Perform spatial moving average for each cell
spatial_averaged = copy.deepcopy(data_by_cell)
smoothing_window = config['settings']['smoothing']['window_size']

# average all coordinates by cell
for cell_type in ['seam_cells', 'annotations']:
    # seam cells should just result in the average warping model
    for cell in spatial_averaged[cell_type].keys():
        # spatial average all the coordinates by time
        coordinates = np.array(spatial_averaged[cell_type][cell])
        # go through each axis
        for dim_idx in range(3):
            coord_dim_data = coordinates[:, dim_idx].copy()
            data_smoothed = moving_average(coord_dim_data, smoothing_window)
            coordinates[:, dim_idx] = data_smoothed

        spatial_averaged[cell_type][cell] = coordinates.tolist()

print('Step 7 Completed.')
# save information as intermediate step in workspace
output_json = spatial_averaged
compiled_json = json.dumps(output_json, sort_keys=True, indent=4)
compiled_json_filepath = os.path.join(
    workspace_folderpath, '7_cell_coordinates_by_timepoint_smoothed.json')
with open(compiled_json_filepath, "w") as f: 
    f.write(compiled_json)
print('Step 7 Data Logged.')

Step 7 Completed.
Step 7 Data Logged.


In [13]:
# STEP 8: Convert to csv in MIPAV format
scale = config['settings']['voxel_to_micron_scale']
labels_on = config['settings']['mipav_output']['labels_on']
# create timepoints array again just to be safe
total_len = config['settings']['interpolation']['total_min'] # in minutes
cell_info_file = config['settings']['mipav_output']['cell_info']
cell_info = None
with open(cell_info_file) as f:
    cell_info = json.load(f)
timepoints = np.arange(0, total_len)

# generate place to put all the files
output_folder = os.path.join(workspace_folderpath, 'output')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
                

for cell_type in ['seam_cells', 'annotations']:
    cells = spatial_averaged[cell_type]

    for cell_name in cells.keys():
        filename_csv = cell_name +'.csv'
        filepath_csv = os.path.join(output_folder, filename_csv)

        coordinates = spatial_averaged[cell_type][cell_name]
        new_table = pd.DataFrame(coordinates) * scale[0]
        new_table.columns = ['x', 'y', 'z']
        new_table.insert(0, "timepoints", timepoints, True)
        # colors
        if cell_name.lower() in cell_info.keys():
            color_timepoints = list(cell_info[cell_name.lower()]['colors'].keys())
            colors = [cell_info[cell_name.lower()]['colors'][color_timepoint] for color_timepoint in color_timepoints]
            colors = np.array(colors)
            color_timepoints = np.array(color_timepoints).astype(float) # because originally string
            
            # color_timepoints_sorted_idx, color_timepoints_sorted = (np.argsort(color_timepoints), np.sort(color_timepoints))
            # colors_sorted = np.array(colors[color_timepoints_sorted_idx]) # get sorted order
            
            # if there's one, just use nearest, otherwise use linear
            if color_timepoints.shape[0] == 1: # use single value
                placeholder_color = np.ones((total_len,))

                for dim_idx, channel in enumerate(['R', 'G', 'B']):
                    # color calculation/interpolation goes here
                    dim_color = colors[0, dim_idx]
                    interped_colors = placeholder_color * dim_color

                    new_table.insert(dim_idx + 4, channel, interped_colors, True)
                
            else:
                color_interp_method = 'linear'
                for dim_idx, channel in enumerate(['R', 'G', 'B']):
                    # color calculation/interpolation goes here
                    interp = interpolate.interp1d(color_timepoints, colors[:, dim_idx], kind=color_interp_method)
                    interped_colors = np.round(interp(timepoints))

                    new_table.insert(dim_idx + 5, channel, interped_colors, True)
                
        else: # set color to white
            placeholder_color = np.ones((total_len,)) * 255
            new_table.insert(4, "R", placeholder_color, True)
            new_table.insert(5, "G", placeholder_color, True)
            new_table.insert(6, "B", placeholder_color, True)
            
        if labels_on:
            new_table.insert(7, "label", np.ones((total_len,)), True)
        else:
            new_table.insert(7, "label", np.zeros((total_len,)), True)
            
        # format each column
        for coord_col in ['x', 'y', 'z']: # as scientific notation
            new_table[coord_col] = new_table[coord_col].map(lambda x: '%.6e' % x)
            
        for color_col in ['R', 'G', 'B', 'label']: # as integers
            new_table[color_col] = new_table[color_col].map(lambda x: '%d' % x)

        new_table.to_csv(filepath_csv, index=False, header=False)
        
full_output_folderpath = os.path.join(os.getcwd(), output_folder)
print('Step 8 Completed.')
print('Output location: {}'.format(full_output_folderpath))

Step 8 Completed.
Output location: Y:\RyanC\model_building_code\workspace\2020_08_24-OD1599_NU\output
